# Data Check

This python note book contains the functionality to check the data saved for or generated by the contact graspnet, to figure out how can I use them.

In [19]:
import numpy as np
from tf.transformations import quaternion_from_matrix as matrix2quaternion
from geometry_msgs.msg import PoseStamped
from tf.transformations import quaternion_from_euler as rpy2quaternion

## For the Input Data

### What should be saved

Here the input data of the example file provided by the repo and what I saved to .npy file are compared to verify if the saved data is correct

<img src="https://i.kym-cdn.com/photos/images/newsfeed/002/046/368/3fa" width="300" height="400" />

In [20]:
reader_test_data = np.load("/home/franka/contact_graspnet/test_data/13.npy", allow_pickle=True)
# print(reader_test_data)

The result shows that the depth should be given in meter, but the encoding method of the realsense camera is 16UC1, which means the depth value is given by mm, so the saved cv::Mat should be multipied with 10^(-3)
 
### What I saved:

Depth image:

In [21]:
reader_my_data_depth_image = np.load("/home/franka/contact_graspnet/depth_image_data/data.npy", allow_pickle=True)
# print(reader_my_data_depth_image)

Point cloud:

In [22]:
reader_my_data_point_cloud = np.load("/home/franka/contact_graspnet/point_cloud_data/data.npy", allow_pickle=True)
# print(reader_my_data_point_cloud)

## For Output Data

### Output of Contact GraspNet

The output data are generated in a .npz file, here they are loaded

In [23]:
output_data = np.load("/home/franka/contact_graspnet/results/predictions_data.npz", allow_pickle=True)

pred_grasps_cam = output_data["pred_grasps_cam.npy"].item()[-1]
scores = output_data["scores.npy"].item()[-1]
contact_pts = output_data["contact_pts.npy"].item()[-1]

<img src="https://media.tenor.com/RXyf_3Ud5FAAAAAC/%E8%AE%A9%E6%88%91%E7%9C%8B%E7%9C%8B.gif" width="600" height="350" />

In [24]:
print(pred_grasps_cam)
# print(pred_grasps_cam.size)

[]


This should be the transformation between the camera_color_optical_link and the grasp pose

In [25]:
print(scores)
# print(scores.size)

[]


In [26]:
print(contact_pts)
# print(contact_pts.size)

[]


## To Use the Output

Now the out put is the potential grasp candidate of the selected object, just to pick the most seccessful one and calculate cooresponding geometry_msgs::PoseStamped

In [27]:
index_max_score = np.argmax(scores)
# print(index_max_score)
T_grasp_cam = np.array(pred_grasps_cam[index_max_score])
print(T_grasp_cam)

ValueError: attempt to get argmax of an empty sequence

In [ ]:
r_cam_grasp = T_grasp_cam[0:3, 0:3]
t_cam_grasp = T_grasp_cam[0:3, 3]
print("Rotation")
print(r_cam_grasp)
print("")
print("Translation")
print(t_cam_grasp)

T_cam_base = np.array([[-0.73383682, -0.27496601,  0.62119016,  0.09599208],
                       [-0.67863248,  0.3380292,  -0.65206919,  0.8253307 ],
                       [-0.03068355, -0.90007219, -0.43465914,  0.73996789],
                       [ 0.,          0.,          0.,          1.        ]])

Rotation
[[-0.7362327   0.5244902   0.42763484]
 [ 0.44395885 -0.1025844   0.89015555]
 [ 0.51074654  0.84521395 -0.15732609]]

Translation
[0.4196524  0.10770498 1.090282  ]


Take translation as a point

In [ ]:
T_grasp_base = np.dot(T_grasp_cam, T_cam_base)